In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re
import pandas as pd
from datetime import datetime

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
df_contracts = spark.read.format("delta").load("abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/contracts/delta/contracts_data_delta")

In [0]:
df_transactions = spark.read.format("delta").load("abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/transactions/delta/accounting_data_delta")

In [0]:
df_employees = spark.read.format("delta").load("abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/employees/delta/employees_data_delta")

In [0]:
df_contracts_grouped = df_contracts.groupBy(['project_id']).agg({'contract_amount': 'sum', 'estimated_costs': 'sum'})

In [0]:
display(df_contracts_grouped)

project_id,sum(contract_amount),sum(estimated_costs)
1004,9.2336025E7,6.5952589E7
1005,9.4685007E7,6.6310934E7
1003,1.29971178E8,9.1441115E7
1002,1.23641866E8,8.8865542E7
1001,1.31961665E8,9.4115967E7


In [0]:
df_transactions_grouped = df_transactions.groupBy(['project_id']).agg({'transaction_amount': 'sum'})

In [0]:
df_employees_grouped = df_employees.groupBy(['project_id']).agg({'total_costs': 'sum'}).orderBy('project_id')

In [0]:
display(df_employees_grouped.limit(10))

project_id,sum(total_costs)
1001,259546
1002,526600
1003,524576
1004,383132
1005,342568


In [0]:
df_joined_contracts_transactions = (df_contracts_grouped.join(df_transactions_grouped, df_contracts_grouped.project_id == df_transactions_grouped.project_id, 'left'))

In [0]:
df_joined_contracts_transactions = df_joined_contracts_transactions.drop(df_transactions_grouped.project_id)

In [0]:
df_combined = df_joined_contracts_transactions.join(df_employees_grouped, df_joined_contracts_transactions.project_id == df_employees_grouped.project_id, 'left')

In [0]:
df_combined = df_combined.drop(df_employees_grouped.project_id)

In [0]:
df_combined_IFRS15 = df_combined.withColumnRenamed('sum(contract_amount)', 'contract_amount').withColumnRenamed('sum(estimated_costs)', 'estimated_costs').withColumnRenamed('sum(transaction_amount)', 'recognised_costs_except_payroll_expenses').withColumnRenamed('sum(total_costs)', 'payroll_expenses')

In [0]:
df_combined_IFRS15 = df_combined_IFRS15.withColumn('costs_incurred_percentage', round(((col('recognised_costs_except_payroll_expenses') + col('payroll_expenses')) / col('estimated_costs')),2))

In [0]:
df_combined_IFRS15 = df_combined_IFRS15.withColumn('recognised_revenue',round((col('contract_amount') * col('costs_incurred_percentage')),0))

In [0]:
df_combined_IFRS15 = df_combined_IFRS15.withColumn('recognised_profit',round((col('recognised_revenue') - col('recognised_costs_except_payroll_expenses') - col('payroll_expenses')),0))

In [0]:
#creating temp excel file to save it in blob storage
IFRS_excel = df_combined_IFRS15.toPandas()
datenow = datetime.now().strftime("%Y%m%d")
tmp_path = f"/tmp/IFRS_excel_{datenow}.xlsx"
IFRS_excel.to_excel(tmp_path, index=False)

In [0]:
blob_storage_path = f"abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/daily_reports_IFRS15/IFRS_15_daily_report_{datenow}.xlsx"
dbutils.fs.cp(f"file:{tmp_path}", blob_storage_path)

True

In [0]:
%sql
CREATE OR REPLACE TABLE df_contracts_grouped
USING DELTA
AS SELECT * FROM delta.`abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/contracts/delta/contracts_data_delta`;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE df_accounts_grouped
USING DELTA
AS SELECT * FROM delta.`abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/transactions/delta/accounting_data_delta`;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE df_employees
USING DELTA
AS SELECT * FROM delta.`abfss://ifrs-project@storagezyesnazarov.dfs.core.windows.net/project-data/employees/delta/employees_data_delta`;`

num_affected_rows,num_inserted_rows
